In [1]:
# 필요한 라이브러리 불러오기
# 데이터를 만들기 위한 Faker 라이브러리 설치

In [2]:
!pip install Faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 46.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import random
from faker import Faker

In [4]:

# 시드 값 설정
seed_value =738
random.seed(seed_value)  # Python 기본 난수 생성기에 대한 시드 설정
np.random.seed(seed_value)  # Numpy 난수 생성기에 대한 시드 설정
fake = Faker()
Faker.seed(seed_value)  # Faker 라이브러리에 대한 시드 설정

# 데이터 생성 함수
def create_sample_data(num_samples=1000, num_frauds=100):
    data = []
    for _ in range(num_samples):
        transaction_number = fake.uuid4()
        transaction_amount = round(random.uniform(5.0, 10000.0), 2)
        is_domestic_ip = random.choice([True, False])
        transaction_ip = fake.ipv4()
        recent_7d_amount = round(random.uniform(5.0, 70000.0), 2)
        recent_7d_payment_methods = random.randint(1, 5)
        account_age_days = random.randint(1, 3650)
        transaction_time = fake.time()
        country_code = fake.country_code()
        currency_code = fake.currency_code()
        customer_age = random.randint(18, 80)
        account_balance = round(random.uniform(0.0, 50000.0), 2)
        num_past_transactions = random.randint(0, 100)
        device_id = fake.uuid4()
        payment_method = random.choice(["credit_card", "debit_card", "paypal", "bank_transfer"])
        is_new_account = random.choice([True, False])
        has_promo_code = random.choice([True, False])
        shipping_address_change = random.choice([True, False])
        num_recent_login_failures = random.randint(0, 10)
        label = 0  # 대부분은 사기가 아님

        data.append([transaction_number, transaction_amount, is_domestic_ip, transaction_ip,
                     recent_7d_amount, recent_7d_payment_methods, account_age_days, transaction_time,
                     country_code, currency_code, customer_age, account_balance, num_past_transactions,
                     device_id, payment_method, is_new_account, has_promo_code, shipping_address_change,
                     num_recent_login_failures, label])

    # 사기 데이터 설정
    fraud_samples = random.sample(data, num_frauds)
    for sample in fraud_samples:
        sample[-1] = 1  # 사기 건으로 라벨 변경

    columns = ["transaction_number", "transaction_amount", "is_domestic_ip", "transaction_ip",
               "recent_7d_amount", "recent_7d_payment_methods", "account_age_days", "transaction_time",
               "country_code", "currency_code", "customer_age", "account_balance", "num_past_transactions",
               "device_id", "payment_method", "is_new_account", "has_promo_code", "shipping_address_change",
               "num_recent_login_failures", "label"]

    return pd.DataFrame(data, columns=columns)

# 샘플 데이터 생성
sample_data = create_sample_data()
sample_data.head()


,transaction_number,transaction_amount,is_domestic_ip,transaction_ip,recent_7d_amount,recent_7d_payment_methods,account_age_days,transaction_time,country_code,currency_code,customer_age,account_balance,num_past_transactions,device_id,payment_method,is_new_account,has_promo_code,shipping_address_change,num_recent_login_failures,label
0,4134ac65-17af-4295-a9ee-2572da24bf01,8521.97,True,180.113.37.42,17833.43,2,569,08:09:41,JM,KZT,73,31870.68,40,239d6aa8-67d8-40ab-91d6-28e19250050d,credit_card,False,True,True,0,0
1,cca31260-0482-493f-ad6d-6470be56ed98,7994.64,False,131.246.97.76,60694.16,3,1713,03:49:40,CD,KRW,70,48163.08,38,8cea40ba-fff7-4f6a-a31d-ef691eee67b5,credit_card,False,True,True,3,1
2,31691814-dbf5-4237-ad62-4495a8e30ee9,6186.20,True,214.90.251.73,48230.44,4,1894,03:11:39,FM,GTQ,64,22340.42,25,658789a9-ec29-40a2-ac55-d79ab06141a3,paypal,True,False,True,6,0
3,5658af95-7262-4a66-baf4-856776546912,3828.94,False,103.110.60.223,13009.76,4,2860,18:37:25,GN,MAD,61,39208.54,68,61f12119-6545-4909-b514-80b5919fa63b,credit_card,False,True,False,0,0
4,2f904bf6-5ff8-434e-82a4-a7a4cc6fa6fb,4278.94,False,34.52.56.109,5958.43,4,1471,04:10:59,IT,ZWD,28,26655.01,54,6d77bba6-00f4-49f0-a19d-f37637e98f66,paypal,True,False,False,9,0


In [5]:
sample_data['label'].value_counts()

label
0    900
1    100
Name: count, dtype: int64

In [6]:
# 샘플 데이터를 위한 가정된 규칙에 따른 분류 및 탐지율 계산 함수
def calculate_detection_rate(df):
    # 규칙 정의
    rules = {
        '고액 거래': df['transaction_amount'] > 5000,
        '다수 결제 수단 사용': df['recent_7d_payment_methods'] >= 3,
        '빈번한 로그인 실패': df['num_recent_login_failures'] >= 3,
        '신규 계정 대량 거래': (df['account_age_days'] <= 30) & (df['transaction_amount'] > 3000),
        '국제 거래 및 고액': (~df['is_domestic_ip']) & (df['transaction_amount'] > 2000),
    }

    # 결과 저장을 위한 리스트
    results = []

    for rule_name, condition in rules.items():
        # 규칙에 해당하는 트랜잭션 필터링
        filtered_transactions = df[condition]
        # 사기 거래와 정상 거래의 수 계산
        num_good = (filtered_transactions['label'] == 0).sum()
        num_bad = (filtered_transactions['label'] == 1).sum()
        # 사기 거래 탐지율 계산
        detection_rate = num_bad / (num_bad + num_good) if num_bad + num_good > 0 else 0
        
        # 결과 추가
        results.append({
            '규칙명': rule_name,
            '정상 거래 수': num_good,
            '사기 거래 수': num_bad,
            '탐지율': detection_rate
        })

    # 결과를 데이터프레임으로 변환
    results_df = pd.DataFrame(results)
    return results_df

# 규칙 적용 및 결과 출력
rule_results = calculate_detection_rate(sample_data)
rule_results


,규칙명,정상 거래 수,사기 거래 수,탐지율
0,고액 거래,437,50,0.102669
1,다수 결제 수단 사용,548,55,0.091211
2,빈번한 로그인 실패,652,75,0.103164
3,신규 계정 대량 거래,6,1,0.142857
4,국제 거래 및 고액,365,32,0.080605


In [7]:
#시각화